In [ ]:
import pandas as pd 
import numpy as np 


In [ ]:
df = pd.read_csv('Result.csv',encoding = 'cp949', index_col= 0)
df.head()

In [ ]:
df.columns

In [ ]:
df['check_상품명'] = df['상품명_x'] == df['상품명_y']
df['check_매입과세구분'] = df['매입과세구분_x'] == df['매입과세구분_y']
df['check_매입형태'] = df['매입형태_x'] == df['매입형태_y']
df['check_배송방식'] = df['배송방식_x'] == df['배송방식_y']
df['check_판매단가'] = df['판매단가_x'] == df['판매단가_y']
df['check_협력사ID'] = df['협력사ID_x'] == df['협력사ID_y']
df['check_협력사명'] = df['협력사명_x'] == df['협력사명_y']




In [ ]:
wrong_price = df[df['check_판매단가'] == False]  # 판매단가_y삭제 
wrong_price[['방송Code','상품명_x', '판매단가_x', '판매단가_y','상품ID']]

In [ ]:
df[df['check_협력사ID'] == False]

In [ ]:
df[df['check_협력사명'] == False]

- 1 & 2 ==> 상품명, 매입과세구분, 매입형태, 배송방식, 판매단가 
- 2 & 3 ==> 협력사ID, 협력사명 

In [ ]:
df.drop(['상품명_y', '매입과세구분_y', '매입형태_y', '배송방식_y', '판매단가_y', '협력사ID_y', '협력사명_y', 'check_상품명', 'check_매입과세구분', 'check_매입형태','check_배송방식', 'check_판매단가', 'check_협력사ID', 'check_협력사명'], axis = 1,inplace = True)
df.columns

In [ ]:
df['check_방송구분'] = df['방송구분'] == df['방송구분.1']
df['check_상품ID'] = df['상품ID'] == df['상품ID.1']
df['check_프로그램명'] = df['프로그램명'] == df['프로그램명.1']

In [ ]:
df[df['check_상품ID'] == False]

In [ ]:
df.drop(['상품ID.1', '프로그램명.1', '방송구분.1'], axis = 1, inplace = True)


In [ ]:
df.columns

In [ ]:
df.drop(['check_방송구분', 'check_상품ID', 'check_프로그램명'], axis = 1, inplace = True)


In [ ]:
df.rename(columns={'상품명_x':'상품명', '매입과세구분_x':'매입과세구분', '매입형태_x' : '매입형태', '배송방식_x':'배송방식', '협력사ID_x': '협력사ID', '협력사명_x':'협력사명'}, inplace = True)

In [ ]:
df.rename(columns = {'판매단가_x' : '판매단가'},inplace =True)

In [ ]:
df.drop('차이', axis = 1, inplace = True)

In [ ]:
df.shape

In [ ]:
df.to_csv('Total1.csv')


In [ ]:
df1 = pd.read_csv('Total1.csv', index_col = 0, parse_dates = ['방송시작시간', '방송종료시간'])
df1

In [ ]:
df1.sort_values(by = '방송시작시간', inplace = True)

In [ ]:
df1[['방송시작시간', '방송종료시간']]

In [ ]:
df1.reset_index(drop = True, inplace = True)

In [ ]:
df1.iloc[:, 20:].isna().sum()

In [ ]:
def is_ger(x):
    if '게릴라' in x:
        return 1
    else:
        return 0 

df1['is_ger'] = df1['상품명'].apply(lambda x:is_ger(x))
df1['is_ger1'] = df1['프로그램명'].apply(lambda x:is_ger(x))
gerrila = df1[df1['is_ger1'] == 1]

In [ ]:
gerrila[['상품명','상품목표주문금액','PD', 'TD', 'MD담당자명', 'SH']]

In [ ]:
gerrila.iloc[:, 20:].isna().sum()

In [ ]:
df1.iloc[:, 20:].isna().sum()

In [ ]:
null_td = df1[df1['TD'].isnull()]

In [ ]:
df1.drop(['is_ger', 'is_ger1'], axis = 1, inplace = True) 


In [ ]:
df1.to_csv('total2.csv')

시간 순 정렬 및 기초데이터 완료

In [ ]:
df2 = pd.read_csv('total2.csv', index_col = 0, parse_dates = ['방송시작시간', '방송종료시간'])

In [ ]:
df2.shape

In [ ]:
df2.columns

In [ ]:
df2.drop(['채널', '프로그램명', 'ARS금액', '브랜드ID', '부서코드', '카테고리', '세분류', '상품방송순번'], axis = 1, inplace = True)


In [ ]:
df2.iloc[:, 10:].info()

In [ ]:
df2['date'] = df2['방송시작시간'].dt.date
df2['hour'] = df2['방송시작시간'].dt.hour
df2['분기'] = df2['방송시작시간'].dt.quarter
df2['요일'] = df2['방송시작시간'].dt.weekday

#### 1. 요일관련 변수 : date, hour, 분기, 요일 생성 && 방송시작, 종료시간 drop

In [ ]:
df2.drop(['방송시작시간', '방송종료시간'], axis = 1, inplace = True)

In [ ]:
zero_goal = df2[df2['상품목표주문금액'] == 0]

In [ ]:
gift = zero_goal[zero_goal['상품주문수량'] == 0]
gift_idx = gift.index

목표주문금액 & 주문수량 = 0 drop

In [ ]:
df2.drop(gift_idx, inplace = True)

In [ ]:
df2.reset_index(drop = True, inplace = True)

In [ ]:
df2

In [ ]:
df2[df2['판매단가'] == 0]['소분류'].value_counts()

- 렌탈서비스, 상조서비스, 소개방송 판매단가 => 판매단가 * 실수량으로 값 대체 

In [ ]:
df2['매출'] = (df2['상품주문금액'] - df2['상품취소금액'])*df2['수수료율'] / 100
df2['실수량'] = df2['상품주문수량'] - df2['상품취소수량']
df2['매출'] = df2['매출'].round(0)

In [ ]:
df2['실제판매단가'] = (df2['상품주문금액'] - df2['상품취소금액'])/(df2['상품주문수량'] - df2['상품취소수량'])
df2['실제판매단가'] = df2['실제판매단가'].round(0)

In [ ]:
df2.columns

In [ ]:
df2[df2['판매단가'] == 0].iloc[:,:20]

In [ ]:
zero_mon = df2[df2['판매단가'] == 0]
real1 = zero_mon[zero_mon['소분류'] == '렌탈서비스']
real2 = zero_mon[zero_mon['소분류'] == '소개방송']
real3 = zero_mon[zero_mon['소분류'] == '상조서비스']


In [ ]:
real1_idx = real1.index
real2_idx = real2.index
real3_idx = real3.index

In [ ]:
real1_value = (real1['상품주문금액'] - real1['상품취소금액'])/(real1['상품주문수량'] - real1['상품취소수량']).round(0)
min(real1_value)

In [ ]:
real2_value = (real2['상품주문금액'] - real2['상품취소금액'])/(real2['상품주문수량'] - real2['상품취소수량']).round(0)
min(real2_value)

In [ ]:
real3_value = (real3['상품주문금액'] - real3['상품취소금액'])/(real3['상품주문수량'] - real3['상품취소수량']).round(0)
min(real3_value)

In [ ]:
df2.loc[real1_idx, '판매단가'] = real1_value
df2.loc[real2_idx, '판매단가'] = real2_value
df2.loc[real3_idx, '판매단가'] = real3_value

In [ ]:
df2.columns

#### total3 = 매출, 실수량, 실제판매단가 추가 및 판매단가 이상치 처리 완료 

In [ ]:
df2.to_csv('total3.csv')

In [ ]:
df3 = pd.read_csv('total3.csv', index_col = 0)
df3

In [ ]:
zero_price = df3[df3['상품목표주문금액'] == 0]

In [ ]:
zero_price['상품목표주문금액'] = zero_price['판매단가'] * zero_price['실수량']

In [ ]:
zero_idx = zero_price.index
zero_value = zero_price['상품목표주문금액'].values

In [ ]:
df3.loc[zero_idx,'상품목표주문금액'] = zero_value

- 상품목표주문금액 처리 완료

In [ ]:
df3.columns

In [ ]:
df3['단위시간영업효율'] = (df3['가중분']/df3['소요분']*100).round(2)
df3

In [ ]:
df3['상품목표주문수량'] = (df3['상품목표주문금액']/df3['판매단가']).round(0)
df3

In [ ]:
df3 = df3.astype({'상품목표주문금액' : 'int'})
df3 = df3.astype({'판매단가' : 'int'})
df3 = df3.astype({'상품목표주문수량' : 'int'})
df3 = df3.astype({'매출' : 'int'})
df3.info()

In [ ]:
df3['금액달성률'] = ((df3['상품주문금액'] - df3['상품취소금액'])/ df3['상품목표주문금액'])*100
df3['금액달성률'] = df3['금액달성률'].round(2)
df3.columns

In [161]:
df3[df3['상품목표주문금액'] == 0].iloc[:, :20]

,방송Code,소요분,가중분,방송구분,상품ID,상품명,매입과세구분,상품목표취급금액,상품목표주문금액,판매단가,수수료율,상품주문수량,상품주문금액,상품취소수량,상품취소금액,매입형태,배송방식,상품소요분,상품가중분,PD
122,101013024611,65.0,117.7,생방송,10247375,[단품] 미넴옴므 허리밴드 기모 정장팬츠,과세,0,0,79000,24.00,2,139328,2,139328,위탁매입,협력사배송,0.0,0.0,강승필
141,101315336152,60.0,78.8,생방송,10247544,[ADRIENNE VITTADINI]리얼 카이만 명함지갑,과세,0,0,128000,25.00,1,116364,1,116364,위탁매입,협력사배송,0.0,0.0,이혁애
590,100561736515,75.0,91.0,생방송,10402608,[단품] PN풍년메가IH28cm프라이팬,과세,0,0,76000,25.50,1,69091,1,69091,위탁매입,협력사배송,0.0,0.0,이효경
875,101691536223,70.0,74.8,생방송,10247100,(단품) [유화연] 윤효 한방기초 4종_3,과세,0,0,75000,25.00,1,62829,1,62829,위탁매입,협력사배송,0.0,0.0,이혁애
958,1016100236805,70.0,74.5,생방송,10411739,[단품] 글램폭스 플로리사 립글로즈 1종,과세,0,0,35000,25.00,1,29955,1,29955,위탁매입,협력사배송,0.0,0.0,이혁애
1147,1016119920440,70.0,74.8,생방송,10063173,[단품]커버퀸 더 레이디 올데이 커버 팩트 본품1개,과세,0,0,35000,26.00,1,31818,1,31818,위탁매입,직택배,0.0,0.0,안수연
1323,1020138123424,60.1,78.3,생방송,10106101,[황풍정]홍삼정올데이수(10ml*30포)_3,과세,0,0,58900,25.50,1,53545,1,53545,위탁매입,협력사배송,0.0,0.0,이혁철
1845,1016192036453,60.0,50.3,생방송,10401657,(S)단품_[더마픽스]콜라겐 골드마스크_3,과세,0,0,52000,10.00,1,47273,1,47273,위탁매입,협력사배송,0.0,0.0,이혁애
1996,1013207936153,60.0,52.6,생방송,10246879,[PGA TOUR]밴드 팔찌,과세,0,0,63000,25.00,3,171819,3,171819,위탁매입,협력사배송,0.0,0.0,이혁애
2126,1007221327684,65.0,111.6,생방송,10242961,[파로마]유로탑 매트리스 Q,과세,0,0,399000,25.00,1,362727,1,362727,위탁매입,협력사배송,0.0,0.0,최성철


In [ ]:
df3['수량달성률'] = ((df3['상품주문수량'] - df3['상품취소수량'])/ df3['상품목표주문수량'])*100
df3['수량달성률'] = df3['수량달성률'].round(2)
df3

#### 단위시간영업효율, 상품목표주문수량, 금액달성률, 수량달성률 생성 완료

In [163]:
df3.iloc[:, 20:].isna().sum()

SH          630
TD          637
MD담당자명        0
부서명           0
협력사ID         0
협력사명          0
브랜드명          0
대분류           0
중분류           0
소분류           0
date          0
hour          0
분기            0
요일            0
매출            0
실수량           0
실제판매단가      904
단위시간영업효율      0
상품목표주문수량      0
금액달성률        48
수량달성률        58
dtype: int64

In [155]:
df3.to_csv('total4.csv')

In [162]:
df3.columns

Index(['방송Code', '소요분', '가중분', '방송구분', '상품ID', '상품명', '매입과세구분', '상품목표취급금액',
       '상품목표주문금액', '판매단가', '수수료율', '상품주문수량', '상품주문금액', '상품취소수량', '상품취소금액',
       '매입형태', '배송방식', '상품소요분', '상품가중분', 'PD', 'SH', 'TD', 'MD담당자명', '부서명',
       '협력사ID', '협력사명', '브랜드명', '대분류', '중분류', '소분류', 'date', 'hour', '분기',
       '요일', '매출', '실수량', '실제판매단가', '단위시간영업효율', '상품목표주문수량', '금액달성률', '수량달성률'],
      dtype='object')

### 수량달성률 및 금액달성률 변수 사용의 적합성 판단

In [168]:
zero_mer = df3[df3['수량달성률'].isnull()]
zero_mer.index

Int64Index([  122,   141,   590,   875,   958,  1147,  1323,  1845,  1996,
             2126,  2127,  3297,  3648,  4857,  6772,  7267,  7687,  7873,
             9508,  9686,  9944, 10157, 10693, 11064, 11802, 12894, 13946,
            14356, 14746, 14841, 16000, 16192, 16234, 16545, 16593, 17377,
            17385, 18379, 18828, 19502, 19505, 19506, 20034, 20162, 20560,
            20709, 21015, 21781, 21785, 21969, 22120, 22323, 22757, 23543,
            24695, 25043, 25044, 25448],
           dtype='int64')

In [167]:
zero_pr = df3[df3['금액달성률'].isnull()]
zero_pr.index

Int64Index([  122,   141,   590,   875,   958,  1147,  1323,  1845,  1996,
             2126,  2127,  3297,  3648,  6772,  7267,  7687,  7873,  9686,
            10157, 10693, 11064, 11802, 12894, 13946, 14356, 14746, 14841,
            16000, 16192, 16234, 16545, 16593, 17377, 18379, 18828, 20034,
            20162, 20560, 21015, 21969, 22120, 22323, 22757, 23543, 24695,
            25043, 25044, 25448],
           dtype='int64')

In [170]:
df4 = pd.read_csv('total_5.csv', index_col=0)

In [172]:
df4.iloc[:, 20:].isna().sum()

SH          630
TD          637
MD담당자명        0
부서명           0
협력사ID         0
협력사명          0
브랜드명          0
대분류           0
중분류           0
소분류           0
date          0
hour          0
분기            0
요일            0
매출            0
실수량           0
단위시간영업효율      0
상품목표주문수량      0
금액달성률        48
수량달성률        58
dtype: int64

In [177]:
check_idx = [4857, 9508, 9944, 17385, 20709, 19502, 19505, 19506, 21781, 21785]
check = df4.loc[check_idx, :]

In [183]:
zero_price = df4[df4['금액달성률'].isnull()]

In [184]:
zero_price[['상품목표주문금액', '상품주문수량', '상품주문금액', '상품취소수량', '상품취소금액','실수량', '상품목표주문수량','금액달성률', '수량달성률']]

,상품목표주문금액,상품주문수량,상품주문금액,상품취소수량,상품취소금액,실수량,상품목표주문수량,금액달성률,수량달성률
122,0,2,139328,2,139328,0,0,NaN,NaN
141,0,1,116364,1,116364,0,0,NaN,NaN
590,0,1,69091,1,69091,0,0,NaN,NaN
875,0,1,62829,1,62829,0,0,NaN,NaN
958,0,1,29955,1,29955,0,0,NaN,NaN
1147,0,1,31818,1,31818,0,0,NaN,NaN
1323,0,1,53545,1,53545,0,0,NaN,NaN
1845,0,1,47273,1,47273,0,0,NaN,NaN
1996,0,3,171819,3,171819,0,0,NaN,NaN
2126,0,1,362727,1,362727,0,0,NaN,NaN


In [179]:
zero_pr[['상품목표주문금액', '상품주문수량', '상품주문금액', '상품취소수량', '상품취소금액','실수량', '상품목표주문수량','금액달성률', '수량달성률']]

,상품목표주문금액,상품주문수량,상품주문금액,상품취소수량,상품취소금액,실수량,상품목표주문수량,금액달성률,수량달성률
122,0,2,139328,2,139328,0,0,NaN,NaN
141,0,1,116364,1,116364,0,0,NaN,NaN
590,0,1,69091,1,69091,0,0,NaN,NaN
875,0,1,62829,1,62829,0,0,NaN,NaN
958,0,1,29955,1,29955,0,0,NaN,NaN
1147,0,1,31818,1,31818,0,0,NaN,NaN
1323,0,1,53545,1,53545,0,0,NaN,NaN
1845,0,1,47273,1,47273,0,0,NaN,NaN
1996,0,3,171819,3,171819,0,0,NaN,NaN
2126,0,1,362727,1,362727,0,0,NaN,NaN


In [185]:
df4.drop(zero_price.index, inplace = True)

In [186]:
df4.reset_index(inplace = True, drop = True)

In [187]:
df4   #48개 drop 

,방송Code,소요분,가중분,방송구분,상품ID,상품명,매입과세구분,상품목표취급금액,상품목표주문금액,판매단가,...,date,hour,분기,요일,매출,실수량,단위시간영업효율,상품목표주문수량,금액달성률,수량달성률
0,1010036178,50.0,52.7,녹화방송,10242138,[비지트인뉴욕콜렉션] 퓨어 캐시미어 롱코트,과세,65645345,117223831,149000,...,2020-01-03,0,1,4,19165087,575,105.40,787,65.40,73.06
1,1019226356,60.0,38.1,녹화방송,10092743,[단품_한삼인] 순홍삼진(50ml*30포)*1박스,과세,23774849,29178755,60000,...,2020-01-03,1,1,4,0,0,63.50,486,0.00,0.00
2,1019126356,60.0,38.1,녹화방송,10092003,[한삼인]순홍삼진(50ml*30포)*7박스+쇼7_2,과세,23774849,29178755,179000,...,2020-01-03,1,1,4,7714665,184,63.50,163,101.69,112.88
3,1040336520,60.0,19.7,재방송,10295865,이상용의 우리밀 크레마롤,과세,24568911,28904601,48900,...,2020-01-03,2,1,4,4507349,435,32.83,591,64.97,73.60
4,1021427468,60.0,16.1,재방송,10113666,국내산 자숙 문어슬라이스 120g*9팩,면세,20131383,24550467,49900,...,2020-01-03,3,1,4,1287452,120,26.83,492,23.84,24.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25603,10462628042184,60.0,94.3,생방송,11179504,■연말특가■제우스커브드 UHD TV 55형,과세,17202608,28200996,449000,...,2021-11-13,21,4,5,6048413,99,157.17,63,142.98,157.14
25604,10462627842184,60.0,94.3,생방송,11179512,■연말특가■제우스커브드 UHD TV 65형,과세,92367433,151422021,649000,...,2021-11-13,21,4,5,19700184,223,157.17,233,86.73,95.71
25605,10462627942174,60.0,94.3,생방송,11179514,2019 제우스 UHD TV 75형,과세,33698544,55243515,1289000,...,2021-11-13,21,4,5,12640635,72,157.17,43,152.54,167.44
25606,10202628144834,60.0,91.6,재방송,11152474,[래오이경제] 흑염소진액 70ml x 120포,과세,154250864,202961664,159000,...,2021-11-13,22,4,5,57514516,1336,152.67,1276,94.46,104.70


In [188]:
df4.drop('수량달성률', axis = 1, inplace = True)

In [189]:
df4.columns

Index(['방송Code', '소요분', '가중분', '방송구분', '상품ID', '상품명', '매입과세구분', '상품목표취급금액',
       '상품목표주문금액', '판매단가', '수수료율', '상품주문수량', '상품주문금액', '상품취소수량', '상품취소금액',
       '매입형태', '배송방식', '상품소요분', '상품가중분', 'PD', 'SH', 'TD', 'MD담당자명', '부서명',
       '협력사ID', '협력사명', '브랜드명', '대분류', '중분류', '소분류', 'date', 'hour', '분기',
       '요일', '매출', '실수량', '단위시간영업효율', '상품목표주문수량', '금액달성률'],
      dtype='object')

In [190]:
df4.to_csv('mid.csv')